### Part 2 SMP ETL Workbook
[Josh King](https://github.com/kingjml), *CPS/CRD/ECCC*, 2021  
[Benoit Montpetit](https://github.com/ecccben), *CPS/CRD/ECCC*, 2024  
[Mike Brady](https://github.com/m9brady), *CPS/CRD/ECCC*, 2024


To parameterize and evaluate the radar analysis we calibrate an empirical model to derive snow density and specific surface area (SSA) from SnowMicroPen profiles collected at TVC. SMP profiles were collected in a cross pattern at each of the sites, including calibration profiles at snow pits. This workbook and the imported scripts build on [King et al. 2020](https://tc.copernicus.org/articles/14/4323/2020/tc-14-4323-2020.html) to address specific issues encountered with Tundra deployment of the SMP.

**Changes and additions from SMP-Sea-Ice**
- *ssa*: New tools to calibrate SSA with IceCube reference measurements
- *ssa*: Initial coefficients for SSA adapted from [Calonne, Richter, et al. 2020](https://tc.copernicus.org/articles/14/1829/2020/tc-14-1829-2020.html)
- *smpfunc*: Scaling of individual layers now uses scipy.interpolation.zoom instead of scipy.signal.resample

In [ ]:
# Establish path from notebook
import sys
sys.path.append("..")
import warnings

# Community imports
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import matplotlib

font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)
plt.rcParams["axes.labelsize"] = 22
plt.rcParams["axes.labelweight"] = 'bold'
plt.rcParams['xtick.labelsize']=16
plt.rcParams['ytick.labelsize']=16

# ECCC imports
import constants
import smpfunc
import tvcfunc

# Init random seed
np.random.seed(constants.RANDOM_SEED) 

# Load reference and meta datasets from Part 1
ref_rho = pd.read_json("../Data/ref_rho.json") # Density reference measurements
ref_ssa = pd.read_json("../Data/ref_ssa.json") # SSA reference measurements
site_meta = pd.read_json("../Data/site_meta.json") # Reconciled SMP site paths

# SMP meta data generation

The following loads SMPmeta data collected in situ and its metadata attaches to the site level data.

In [ ]:
smp_meta_df = pd.DataFrame()
for idx, site in site_meta.iterrows():
    smp_meta = pd.read_csv(site['smp_meta'][0]).assign(site = site['site'], path = None)
    smp_meta.columns  = ['position', 'file', 'probe', 'pen', 'notes', 'site', 'path']
    smp_meta.replace(['-99999', -99999], np.nan, inplace = True)
    smp_meta['file'] = smp_meta['file'].astype('Int64')
    
    for idx, profile in smp_meta.iterrows():
        smp_file_match = [file for file in site['pnt_files'] if str(profile.file) in file]
        if len(smp_file_match)==1:
            smp_meta.iloc[idx, smp_meta.columns.get_loc('path')] = smp_file_match[0]
            
    smp_meta_df = pd.concat([smp_meta_df,smp_meta], ignore_index = True)
smp_meta_df.head()

smp_meta_df.to_json("../Data/smp_meta.json")

### Evaluate inter-site profiles to find a best match with reference data
We collected 2 or more SMP profiles at each snow pit. Rather than assign the profile we thought was closest the following establishes a best match against the reference measurements. First guess values for density and SSA are taken from [King et al. 2020](https://tc.copernicus.org/articles/14/4323/2020/tc-14-4323-2020.html) and [Calonne, Richter, et al. 2020](https://tc.copernicus.org/articles/14/1829/2020/tc-14-1829-2020.html), respectively.

#### Find best profile match with density and SSA first guess 
- Load associated pit SMP profiles, estimate density and SSA from first guess parameters
- Scale each SMP density profile with 500 random members
- Compare scaling candidates against reference measurements and score
- Select best scaling candidate for each profile and report in metadata

The loop scores the SMP derived properties at each snow pit against observations. Most of the time the profile we thought was the best candidate is selected but at sites with >4 profiles available, there can a better match where layer structures are drastically different. This is due to hummock and vegetation interactions over sub-meter distances. This can also happen where the reference profile was of poor quality and the neighbouring profile is a better match.

This process is brute force and slow. I've provided data intermediaries so this step can be skipped if desired!

In [ ]:
# Isolate the available calibration files for each site
cal_profiles = smp_meta_df.loc[[('SSA' in x )| ('RHO' in x) for x in smp_meta_df['position']]]
print("Calibration profiles available at {} sites".format(len(cal_profiles.site.unique())))

In [ ]:
# Find the best profile match for SMP density and SSA calibrations
site_meta = pd.DataFrame(); site_result = pd.DataFrame()
num_tests = 500
for site in cal_profiles.site.unique():
    smp_df = cal_profiles.loc[cal_profiles['site'] == site] # SMP profiles at the current site
    smp_data = [smpfunc.load_smp_data(smp_file, constants.WINDOW_SIZE) for smp_file in smp_df['path'].tolist()]
    smp_density = [smpfunc.calc_density(profile, constants.KING_COEFF, True) for profile in smp_data]
    smp_ssa = [smpfunc.calc_ssa(profile, constants.CALONNE_COEFF, True) for profile in smp_data]
    
    for idx, profile in enumerate(smp_density):
        meta, result = smpfunc.eval_scaling(profile, 
                                            ref_rho.loc[ref_rho.site.str.contains(site)].copy(), 
                                            layer_h = constants.L_HEIGHT,
                                            num_tests = num_tests)
        
        site_meta = pd.concat([site_meta,meta.assign(site = site, 
                                                 ref_type = 'density', 
                                                 smp_file = smp_df.iloc[idx]['file'])], ignore_index=True)
        
        result['meta_index'] = meta['index'].values[0]
        
        site_result = pd.concat([site_result,result.assign(site = site, 
                                                       ref_type = 'density', 
                                                       smp_file = smp_df.iloc[idx]['file'])], ignore_index=True)
    
    for idx, profile in enumerate(smp_ssa):
        meta, result = smpfunc.eval_scaling(profile, 
                                            ref_ssa.loc[ref_ssa.site.str.contains(site)].copy(), 
                                            layer_h = constants.L_HEIGHT,
                                            num_tests = num_tests)
        
        site_meta = pd.concat([site_meta,meta.assign(site = site, 
                                                 ref_type = 'ssa', 
                                                 smp_file = smp_df.iloc[idx]['file'])], ignore_index=True)
        
        result['meta_index'] = meta['index'].values[0]
        
        site_result = pd.concat([site_result,result.assign(site = site, 
                                                       ref_type = 'ssa', 
                                                       smp_file = smp_df.iloc[idx]['file'])], ignore_index=True)

In [ ]:
# Separate results for density and SSA
site_meta_dens = site_meta.loc[site_meta['ref_type'] == 'density']
site_result_dens = site_result.loc[site_result['ref_type'] == 'density']

site_meta_ssa = site_meta.loc[site_meta['ref_type'] == 'ssa']
site_result_ssa = site_result.loc[site_result['ref_type'] == 'ssa']

# Extract the best match for each site
first_match_meta_dens = site_meta_dens.sort_values(['rmse'], ascending=[True]).drop_duplicates(['site'])
first_match_result_dens = site_result_dens.loc[site_result_dens['smp_file'].isin(first_match_meta_dens['smp_file'].values)]

first_match_meta_ssa = site_meta_ssa.sort_values(['rmse'], ascending=[True]).drop_duplicates(['site'])
first_match_result_ssa = site_result_ssa.loc[site_result_ssa['smp_file'].isin(first_match_meta_ssa['smp_file'].values)]

# New methods to extract SMP derived properties

In [King et al. 2020](https://tc.copernicus.org/preprints/tc-2019-305/) we scaled profiles of L and F directly but doing this introduced noise when calibrating regression coefficients and produced interpolation errors at layer boundaries. For TVC we first find the best scaling parameters and then create a translation function between the original and scaled heights. This way we can sample the density cutter measurements against the original SMP profiles without interpolation. 

An example of the new approach is shown below with 10k member scaling:

In [ ]:
# Example site
site = 'RS28'
site_df = first_match_meta_dens.loc[first_match_meta_dens.site.str.contains(site)]
site_ref = ref_rho.loc[ref_rho.site.str.contains(site)].copy()
smp_file = cal_profiles.loc[cal_profiles['file'] == site_df['smp_file'].values[0]]['path'].values[0]
smp_data = smpfunc.load_smp_data(smp_file, constants.WINDOW_SIZE)
smp_model_dens = smpfunc.calc_density(smp_data, constants.KING_COEFF, True)

meta, result = smpfunc.eval_scaling(smp_model_dens, 
                                    site_ref, 
                                    num_tests = 10000,
                                    layer_h = constants.L_HEIGHT)

In [ ]:
# Create a lookup table of original height to scaled height
orig_dist, scaled_dist = smpfunc.scale_profile(meta['scale_coeff'][0],
                                               smp_model_dens['rel_height'].values,
                                               smp_model_dens['rel_height'].values,
                                               l_resample = constants.L_HEIGHT)

# Grab all the scaled heights within the cutter height
scaled_cutter_height = [scaled_dist[np.abs(scaled_dist - x)<=constants.CUTTER_SIZE] for x in site_ref['rel_height'].tolist()]

# Extract the mean within the scaled cutter bounds
scaled_mean = pd.DataFrame()
for cutter_idx in np.arange(0, len(scaled_cutter_height)):
    scaled_mean = pd.concat([scaled_mean,pd.DataFrame(smp_model_dens.loc[(smp_model_dens['rel_height']>=scaled_cutter_height[cutter_idx].min()) & \
                  (smp_model_dens['rel_height']<=scaled_cutter_height[cutter_idx].max())].mean()).T], ignore_index=True)

In [ ]:
scaled_dist

In [ ]:
orig_dist

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.scatter(orig_dist, scaled_dist, color = 'k', s = 5)
ax1.plot([0, 500], [0, 500], 'k-', lw=1, alpha = 0.5)
ax1.set_xlim(0,500)
ax1.set_ylim(0,500)

ax1.set_xlabel('Original distance (mm)')
ax1.set_ylabel('Scaled distance (mm)')

ax2.scatter(scaled_mean['smp_val'], site_ref['ref_val'], color = 'k', label = "Original")
ax2.scatter(result['mean_samp'], site_ref['ref_val'], color = 'r', label = "Scaled")
ax2.plot([100, 500], [100, 500], 'k-', lw=1, alpha = 0.5)
ax2.set_xlabel('SMP Density ($kg \\cdot m^{-3}$)')
ax2.set_ylabel('Pit Density ($kg \\cdot m^{-3}$)')

ax2.set_xlim(100,500)
ax2.set_ylim(100,500)
ax2.legend()

<center><img src="../Figures/Part_2_SMP_ETL_Fig1.png" height="500px"></center>

<center>Figure: Example of original vs scaled distance and calibrated SMP density vs pit density</center>

# Extract metrics for all SSA calibration profiles

Use the best SMP profile match to extract SMP derived metrics where IceCube measurements are available

In [ ]:
smp_data

In [ ]:
scaled_result_ssa = pd.DataFrame()
for idx, row in first_match_meta_ssa.iterrows():
    site_ref = ref_ssa.loc[ref_ssa.site.str.contains(row.site)].copy()
    smp_file = cal_profiles.loc[cal_profiles['file'] == row['smp_file']]['path'].values[0]
    smp_data = smpfunc.load_smp_data(smp_file, constants.WINDOW_SIZE)
    smp_model_ssa = smpfunc.calc_ssa(smp_data, constants.CALONNE_COEFF, True)
    
    meta, result = smpfunc.eval_scaling(smp_model_ssa, 
                                        site_ref, 
                                        num_tests = 10000,
                                        layer_h = constants.L_HEIGHT)
    
    # Create a lookup table of origional height to best scaling
    dist_lookup = smpfunc.scale_profile(meta['scale_coeff'][0],
                                smp_model_ssa['rel_height'].values,
                                smp_model_ssa['rel_height'].values,
                                50)
    
    scaled_cutter_height = [dist_lookup[1][np.abs(dist_lookup[0] - x)<=constants.CUTTER_SIZE] for x in site_ref['rel_height'].tolist()]
    
    scaled_mean = pd.DataFrame()
    for cutter_idx in np.arange(0, len(scaled_cutter_height)):
        if scaled_cutter_height[cutter_idx].shape[0]>2:
            scaled_mean = pd.concat([scaled_mean,pd.DataFrame(smp_model_ssa.loc[(smp_model_ssa['rel_height']>=scaled_cutter_height[cutter_idx].min()) & \
                              (smp_model_ssa['rel_height']<=scaled_cutter_height[cutter_idx].max())].mean()).T], ignore_index=True)
        else: # Catch samples that are outside of the profile
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=DeprecationWarning)
                scaled_mean = pd.concat([scaled_mean,pd.DataFrame()], ignore_index=True)
                
    profile_result = pd.concat([scaled_mean, site_ref.rename(columns={'rel_height':'site_rel_height'}).reset_index(drop = True)], axis=1, sort=False)
    scaled_result_ssa = pd.concat([scaled_result_ssa,profile_result], ignore_index = True)

In [ ]:
# Drop NaN results and save to json
scaled_clean_ssa = scaled_result_ssa.dropna()
scaled_clean_ssa.to_json("../Data/Scaled_SMP_ssa.json")

# Calculate RMSE for each profile, remove profiles with errors > 3-sigma
profile_rmse_ssa = scaled_clean_ssa.groupby('site').apply(lambda x: tvcfunc.calc_rmse(x))
result_ssa = scaled_clean_ssa.loc[scaled_clean_ssa['site'].isin(profile_rmse_ssa.loc[profile_rmse_ssa < profile_rmse_ssa.std()*3].index.values)]

# Extract metrics for all density calibration profiles

Use the best SMP profile match to extract SMP derived metrics where density cutter measurements are available

In [ ]:
scaled_result_dens = pd.DataFrame()
for idx, row in first_match_meta_dens.iterrows():
    site_ref = ref_rho.loc[ref_rho.site.str.contains(row.site)].copy()
    smp_file = cal_profiles.loc[cal_profiles['file'] == row['smp_file']]['path'].values[0]
    smp_data = smpfunc.load_smp_data(smp_file, constants.WINDOW_SIZE)
    smp_model_rho = smpfunc.calc_density(smp_data, constants.KING_COEFF, True)

    meta, result = smpfunc.eval_scaling(smp_model_rho, 
                                        site_ref, 
                                        num_tests = 10000,
                                        layer_h = constants.L_HEIGHT)
    
    # Create a lookup table of origional height to best scaling
    dist_lookup = smpfunc.scale_profile(meta['scale_coeff'][0],
                                smp_model_rho['rel_height'].values,
                                smp_model_rho['rel_height'].values,
                                50)

    scaled_cutter_height = [dist_lookup[1][np.abs(dist_lookup[0] - x)<=constants.CUTTER_SIZE] for x in site_ref['rel_height'].tolist()]

    scaled_mean = pd.DataFrame()
    for cutter_idx in np.arange(0, len(scaled_cutter_height)):
        if scaled_cutter_height[cutter_idx].shape[0]>2:
            scaled_mean = pd.concat([scaled_mean,pd.DataFrame(smp_model_rho.loc[(smp_model_rho['rel_height']>=scaled_cutter_height[cutter_idx].min()) & \
                              (smp_model_rho['rel_height']<=scaled_cutter_height[cutter_idx].max())].mean()).T], ignore_index=True)
        else: # Catch samples that are outside of the profile
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=DeprecationWarning)
                scaled_mean = pd.concat([scaled_mean,pd.DataFrame()], ignore_index=True)
                
    profile_result = pd.concat([scaled_mean, site_ref.rename(columns={'rel_height':'site_rel_height'}).reset_index(drop = True)], axis=1, sort=False)
    scaled_result_dens = pd.concat([scaled_result_dens,profile_result], ignore_index = True)

In [ ]:
# Drop NaN results and save to pickle
scaled_clean_dens = scaled_result_dens.dropna()
scaled_clean_dens.to_json("../Data/Scaled_SMP_dens.json")

# Calculate RMSE for each profile, remove profiles with errors > 3-sigma
profile_rmse_dens = scaled_clean_dens.groupby('site').apply(lambda x: tvcfunc.calc_rmse(x))
result_dens = scaled_clean_dens.loc[scaled_clean_dens['site'].isin(profile_rmse_dens.loc[profile_rmse_dens < profile_rmse_dens.std()*3].index.values)]

# Model calibration Density

Following the same procedures as King et al. 2020 we recalibrate the density model using OLS regression with 10-folds

In [ ]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=constants.RANDOM_SEED)
rmse = []; error = []; r = []; params = None

# Split the dataset into 10 roughly equal groups, 
# train on all but one test group
for train_idx, test_idx in k_fold.split(result_dens):
    train = result_dens.iloc[train_idx]
    test = result_dens.iloc[test_idx]
    
    model_rho = ols("ref_val ~ np.log(force_median) + np.log(force_median) * l", train).fit()
    predict_rho = model_rho.predict(exog=dict(force_median=test['force_median'], l=test['l']))
    
    rmse = np.append(rmse, np.sqrt(np.mean((predict_rho - test['ref_val'])**2)))
    r = np.append(r,np.corrcoef(predict_rho, test['ref_val'])[1][0])
    error = np.append(error, predict_rho - test['ref_val'])
    
    if params is None:
        params = model_rho.params.values
    else:
        params =  np.vstack((params, model_rho.params.values))
        
model_ols_tvc = [np.round(params[:,0].mean(),2), np.round(params[:,1].mean(),2),
              np.round(params[:,3].mean(),2), np.round(params[:,2].mean(),2)]
var_coeffs = [np.round(params[:,0].std(),2), np.round(params[:,1].std(),2),
              np.round(params[:,3].std(),2), np.round(params[:,2].std(),2)]

n_obs = len(result_dens)
rmse = rmse.mean()
bias = error.mean()
r2 = r.mean()**2

print(model_ols_tvc)
print('N: %i' % n_obs)
print('RMSE: %0.1f' % rmse)
print('RMSE percent: %0.2f' % np.round(rmse/result_dens.ref_val.mean(),2))
print('bias: %0.1f' % bias)
print('R$^2$: %0.2f' % r2)

filename = '../Data/density_ols_coeffs.npy'
np.save(filename, model_ols_tvc)

[307.76, 53.81, -44.24, -64.8]  
N: 646  
RMSE: 31.0  $kg\cdot m^{-3}$  
RMSE percent: 0.12  \%  
bias: -0.0  $kg\cdot m^{-3}$  
$R^2$: 0.81  


In [ ]:
# Apply the new coeffs to estimate density
tvc_dens_ols = model_ols_tvc[0]+\
              (model_ols_tvc[1]*np.log(result_dens['force_median']))+ \
              (model_ols_tvc[2]*np.log(result_dens['force_median'])*result_dens['l'])+ \
               model_ols_tvc[3]*result_dens['l']

In [ ]:
line_start_a = 100
line_end_a = 500
point_size = 20

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
f.subplots_adjust(hspace=.25)
ax1.set_xlim(line_start_a,line_end_a)
ax1.set_ylim(line_start_a,line_end_a)

ax1.scatter(result_dens['ref_val'], tvc_dens_ols, 
            s = point_size, color ='black', zorder = 1000)
ax1.plot([line_start_a, line_end_a], [line_start_a, line_end_a], 
         '-', color = 'gray' ,alpha= 0.8, zorder = 500)
ax1.set_xlabel('Density Cutter $\\rho_{snow}$ ($kg \\cdot m^{-3}$)')
ax1.set_ylabel("SMP $\\rho_{snow}$ ($kg \\cdot m^{-3}$)")

ax1.text(350, 115,'N: %i \nRMSE: %i $kg \\cdot m^{-3}$\nR$^2$: %0.2f'%(n_obs, rmse, r2))

hist_bins = np.arange(100, 500, 25)
ax2.hist(result_dens['ref_val'], bins = hist_bins, density = True, color = 'dimgrey', label = 'Density\nCutter', edgecolor='k')
ax2.hist(tvc_dens_ols, bins = hist_bins, density = True, color = 'aqua', alpha = 0.5, label = 'SMP', edgecolor='k')
ax2.legend()
ax2.set_xlabel('$\\rho_{snow}$ ($kg\\cdot m^{-3}$)')

f.tight_layout()

<center><img src="../Figures/Figure6.png" height="500px"></center>

<center>Figure 6 of <a href="https://doi.org/10.5194/egusphere-2024-651">Montpetit et al. (2024)</a>: Results of calibrated SMP snow density measurements</center>

# Model calibration SSA

Here we calibrate the regression model suggested by [Calonne, Richter, et al. 2020](https://tc.copernicus.org/articles/14/1829/2020/tc-14-1829-2020.html) with modifications.

In [ ]:
colours = {'F':'#ADD8E6', 'H':'#0000FF', 'M':'k', 'R':'#FFB6C1', 'N': '#00FF00'}

f, (ax1, ax2) = plt.subplots(1, 2, sharey=False, figsize=(15,10))
ax1.scatter(np.log(result_ssa['force_median']), np.log(result_ssa['ref_val']), s =4, c= result_ssa['grain_type'].map(colours))
ax2.scatter(np.log(result_ssa['l']), np.log(result_ssa['ref_val']), s =4, c= result_ssa['grain_type'].map(colours))
ax1.set_ylabel('$ln(SSA_{IC}$) [m$^2$ kg$^{-1}$]')
ax1.set_xlabel(r'$ln(\tilde{F})$ [N]')
ax2.set_xlabel(r'$L$ [mm]')

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Precip.',
                          markerfacecolor='#00FF00', markersize=12),
                  Line2D([0], [0], marker='o', color='w', label='Rounded',
                          markerfacecolor='#FFB6C1', markersize=12),
                  Line2D([0], [0], marker='o', color='w', label='Faceted',
                          markerfacecolor='#ADD8E6', markersize=12),
                  Line2D([0], [0], marker='o', color='w', label='Depth hoar',
                          markerfacecolor='#0000FF', markersize=12),
                  Line2D([0], [0], marker='o', color='w', label='Mixed',
                          markerfacecolor='k', markersize=12)]

ax2.legend(handles=legend_elements,loc='center', bbox_to_anchor=(1.25, 0.5), ncol=1)

<center><img src="../Figures/Part_2_SMP_ETL_Fig2.png" height="500px"></center>

<center>Figure: Example SMP parameter space color coded by grain type</center>

In [ ]:
# Apply log-log model
k_fold = KFold(n_splits=10, shuffle=True, random_state=constants.RANDOM_SEED)
rmse = []; error = []; r = []; params = None
# Split the dataset into 10 roughly equal groups, 
# train on all but one test group
for train_idx, test_idx in k_fold.split(result_ssa):
    train = result_ssa.iloc[train_idx]
    test = result_ssa.iloc[test_idx]

    model_ssa = ols("np.log(ref_val) ~ np.log(l) + np.log(force_median)", train).fit()
    predict_ssa = model_ssa.predict(exog=dict(force_median=test['force_median'], l=test['l']))
    
    rmse = np.append(rmse, np.sqrt(np.mean((np.exp(predict_ssa) - test['ref_val'])**2)))
    r = np.append(r,np.corrcoef(np.exp(predict_ssa), test['ref_val'])[1][0])
    error = np.append(error, np.exp(predict_ssa) - test['ref_val'])
    
    if params is None:
        params = model_ssa.params.values
    else:
        params =  np.vstack((params, model_ssa.params.values))
        
model_ols_tvc = [np.round(params[:,0].mean(),3), np.round(params[:,1].mean(),3),
              np.round(params[:,2].mean(), 3)]
var_coeffs = [np.round(params[:,0].std(),2), np.round(params[:,1].std(),2),
              np.round(params[:,2].std(),2)]

n_obs = len(result_ssa)
rmse = rmse.mean()
bias = error.mean()
r2 = r.mean()**2

                       
print(model_ols_tvc)
print('N: %i' % n_obs)
print('RMSE: %0.1f' % rmse)
print('RMSE percent: %0.3f' % np.round(rmse/result_ssa.ref_val.mean(),2))
print('bias: %0.1f' % bias)
print('R$^2$: %0.2f' % r2)

filename = '../Data/ssa_ols_coeffs.npy'
np.save(filename, model_ols_tvc)

[2.371, -0.699, -0.06]  
N: 627  
RMSE: 5.0  
RMSE percent: 0.290  
bias: -0.6  
R$^2$: 0.69  


In [ ]:
tvc_ssa_ols = np.exp(model_ols_tvc[0] + (model_ols_tvc[1]*np.log(result_ssa['l'])) + \
                    (model_ols_tvc[2]*np.log(result_ssa['force_median'])))

line_start_a = -5
line_end_a = 60
point_size = 20

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))


ax1.scatter(result_ssa['ref_val'], result_ssa['smp_val'], 
            s = point_size, color ='darkgrey', zorder = 500)

ax1.scatter(result_ssa['ref_val'], tvc_ssa_ols, 
            s = point_size, color ='k', zorder = 1000)


ax1.set_xlim(line_start_a,line_end_a)
ax1.set_ylim(line_start_a,line_end_a)

ax1.plot([line_start_a, line_end_a], [line_start_a, line_end_a], 
         '-', color = 'gray' ,alpha= 0.8, zorder = 500)

ax1.set_xlabel('IceCube SSA ($m^2 \\cdot kg^{-1})$')
ax1.set_ylabel('SMP SSA ($m^2 \\cdot kg^{-1}$)')
ax2.set_xlabel('SSA ($m^2 \\cdot kg^{-1}$)')


hist_bins = np.arange(0, 70, 2)
ax2.hist(tvc_ssa_ols, bins = hist_bins, density = True, color = 'k', alpha = 1, label = 'King-TVC', edgecolor='k')
ax2.hist(result_ssa['smp_val'], bins = hist_bins, density = True, color = 'darkgrey',alpha = 0.5, label = 'Calonne2020', edgecolor='k')
ax2.hist(result_ssa['ref_val'], bins = hist_bins, density = True, color = 'aqua', label = 'IceCube', edgecolor='k', alpha=0.5)
ax2.legend()
ax2.text(35, 0.045,'King-TVC \nN: %i \nRMSE: %0.1f $m^2 \\cdot kg^{-1}$\nR$^2$: %0.2f'%(n_obs, rmse, r2))
ax2.text(35, 0.020,'Calonne2020 \nN: %i \nRMSE: %0.1f $m^2 \\cdot kg^{-1}$\nR$^2$: %0.2f'%(n_obs, np.sqrt(np.nanmean((result_ssa['ref_val']-result_ssa['smp_val'])**2)), 
                                                                                           r2_score(result_ssa['ref_val'], result_ssa['smp_val'])))

<center><img src="../Figures/Figure7.png" height="500px"></center>

<center>Figure 7 of <a href="https://doi.org/10.5194/egusphere-2024-651">Montpetit et al. (2024)</a>: Example of original vs scaled distance and calibrated SMP density vs pit density</center>

In [ ]:
bins = np.arange(-30, 30, 1)
plt.hist(result_ssa['ref_val'] - tvc_ssa_ols, 
         bins = bins, color = 'grey', density = True)
plt.xlabel('Error ($m^2 \\cdot kg^{-1}$)')

<center><img src="../Figures/Part_2_SMP_ETL_Fig3.png" height="500px"></center>

<center>Figure: Error distribution of the SMP SSA vs the IceCube SSA measurements</center>